In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/02/28 17:25:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/28 17:25:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark.version

'3.0.3'

In [7]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-02-28 17:30:21--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.138.161
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.138.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  12.4MB/s    in 60s     

2022-02-28 17:31:22 (11.7 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [8]:
from pyspark.sql import types

In [9]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [10]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [11]:
df = df.repartition(24)

In [12]:
df.write.parquet('fhvhv/2021/02/')

go to fhvhv/2021/02 and run ls -lh

In [37]:
df = spark.read.parquet('fhvhv/2021/02/')

In [31]:
df.count()

11613942

In [38]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null|
|           HV0003|              B02871|2021-02-03 18:34:17|2021-02-03 18:57:12|         235|          60|   null|
|           HV0003|              B02869|2021-02-04 07:25:09|2021-02-04 07:30:34|          55|          55|   null|
|           HV0003|              B02836|2021-02-04 23:15:27|2021-02-04 23:34:29|

In [41]:
df.filter((df.pickup_datetime >= '2021-02-15 00:00:00') & (df.pickup_datetime <= '2021-02-15 23:59:59')).count()

367170

In [42]:
df.registerTempTable('fhvhv_02')

In [54]:
spark.sql("""
SELECT 
    pickup_datetime, 
    dropoff_datetime, 
    DATEDIFF(dropoff_datetime, pickup_datetime) AS diff_days 
    FROM fhvhv_02
""").show()

+-------------------+-------------------+---------+
|    pickup_datetime|   dropoff_datetime|diff_days|
+-------------------+-------------------+---------+
|2021-02-06 01:18:35|2021-02-06 01:40:34|        0|
|2021-02-05 07:13:06|2021-02-05 07:31:56|        0|
|2021-02-04 16:56:52|2021-02-04 17:21:36|        0|
|2021-02-03 18:34:17|2021-02-03 18:57:12|        0|
|2021-02-04 07:25:09|2021-02-04 07:30:34|        0|
|2021-02-04 23:15:27|2021-02-04 23:34:29|        0|
|2021-02-05 07:45:49|2021-02-05 08:05:03|        0|
|2021-02-03 17:33:59|2021-02-03 17:47:14|        0|
|2021-02-03 09:33:06|2021-02-03 09:45:59|        0|
|2021-02-02 12:58:46|2021-02-02 13:06:20|        0|
|2021-02-05 22:25:20|2021-02-05 22:28:27|        0|
|2021-02-04 19:57:44|2021-02-04 20:05:24|        0|
|2021-02-02 08:43:57|2021-02-02 08:53:46|        0|
|2021-02-04 05:51:50|2021-02-04 06:21:25|        0|
|2021-02-05 16:13:02|2021-02-05 16:27:48|        0|
|2021-02-04 17:15:28|2021-02-04 17:24:46|        0|
|2021-02-04 

In [56]:
from pyspark.sql.functions import *

In [60]:
df2=df.withColumn('from_timestamp',to_timestamp(col('pickup_datetime')))\
  .withColumn('end_timestamp', to_timestamp(col('dropoff_datetime')))\
  .withColumn('DiffInSeconds',col("end_timestamp").cast("long") - col('from_timestamp').cast("long"))

In [77]:
df2_sorted = df2.orderBy('DiffInSeconds', ascending=False)

In [81]:
df2_sorted.head(1)

[Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime=datetime.datetime(2021, 2, 11, 13, 40, 44), dropoff_datetime=datetime.datetime(2021, 2, 12, 10, 39, 44), PULocationID=247, DOLocationID=41, SR_Flag=None, from_timestamp=datetime.datetime(2021, 2, 11, 13, 40, 44), end_timestamp=datetime.datetime(2021, 2, 12, 10, 39, 44), DiffInSeconds=75540)]

In [90]:
spark.sql("""
SELECT 
    dispatching_base_num, count(1)
    FROM fhvhv_02
    group by dispatching_base_num
""").show()

+--------------------+--------+
|dispatching_base_num|count(1)|
+--------------------+--------+
|              B02876|  215693|
|              B03136|    1741|
|              B02877|  198938|
|              B02869|  429720|
|              B02883|  251617|
|              B02835|  189031|
|              B02884|  244963|
|              B02880|  115716|
|              B02878|  305185|
|              B02836|  128978|
|              B02872|  882689|
|              B02512|   41043|
|              B02867|  200530|
|              B02866|  311089|
|              B02871|  312364|
|              B02889|  138762|
|              B02844|    3502|
|              B02510| 3233664|
|              B02888|  169167|
|              B02682|  303255|
+--------------------+--------+
only showing top 20 rows



In [91]:
df_zones = spark.read.parquet('zones/')

In [92]:
df_zones.registerTempTable('zones')

In [97]:
df_zones.head()

Row(LocationID='1', Borough='EWR', Zone='Newark Airport', service_zone='EWR')

In [102]:
spark.sql("""
select
  concat(zpu.Zone, '/', zdo.Zone),
  count(1) as trip_count
from
    fhvhv_02 fhv join zones zpu
        on fhv.PULocationID = zpu.LocationID
    join zones zdo
        on fhv.DOLocationID = zdo.LocationID
group by 1
order by trip_count desc
""").show()

+---------------------+----------+
|concat(Zone, /, Zone)|trip_count|
+---------------------+----------+
| East New York/Eas...|     45041|
| Borough Park/Boro...|     37329|
|    Canarsie/Canarsie|     28026|
| Crown Heights Nor...|     25976|
|  Bay Ridge/Bay Ridge|     17934|
| Jackson Heights/J...|     14688|
|      Astoria/Astoria|     14688|
| Central Harlem No...|     14481|
| Bushwick South/Bu...|     14424|
| Flatbush/Ditmas P...|     13976|
| South Ozone Park/...|     13716|
| Brownsville/Brown...|     12829|
|       JFK Airport/NA|     12542|
| Prospect-Lefferts...|     11814|
| Forest Hills/Fore...|     11548|
| Bushwick North/Bu...|     11491|
| Bushwick South/Bu...|     11487|
| Crown Heights Nor...|     11462|
| Crown Heights Nor...|     11342|
| Prospect-Lefferts...|     11308|
+---------------------+----------+
only showing top 20 rows

